In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

In [3]:
pdbs_to_be_tested = ["1BG3", "2PGI", "4PFK", "4ALD", "3YPI", "3GPD",
                    "13PK", "1BQ3", "7ENL", "1A3W"]

In [4]:
molsys = msm.convert('1A6M')

/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:33: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)


In [7]:
alt = msm.build.get_atoms_with_alternate_locations(molsys)

/home/diego/Myopt/miniconda3/envs/MolSysMT@dprada_3.9/lib/python3.9/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/home/diego/Myopt/miniconda3/envs/MolSysMT@dprada_3.9/lib/python3.9/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/home/diego/Myopt/miniconda3/envs/MolSysMT@dprada_3.9/lib/python3.9/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/home/diego/Myopt/miniconda3/envs/MolSysMT@dprada_3.9/lib/python3.9/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=co

In [11]:
msm.get(molsys, element='atom', selection=alt[0], alternate_location=True, occupancy=True)

[array(['A', 'B'], dtype=object), array([0.57, 0.43])]

In [ ]:
alternate_location, occupancy = msm.get(molsys, element='atom', alternate_location=True, occupancy=True)

In [ ]:
As = np.where(alternate_location=='A')

In [ ]:
As

In [ ]:
atom_name, group_id, chain_id = msm.get(molsys, element='atom', selection=As[0][0], atom_name=True, group_id=True, chain_id=True)

In [ ]:
chain_id

In [ ]:
msm.select(molsys, selection='atom_name==@atom_name and group_id==@group_id and chain_id==@chain_id')

In [ ]:
As[0][0]

In [ ]:
msm.info(molsys, element='atom', indices=As[0][0], output='short_string')